# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [177]:
problem = '''You are in charge of the supply purchasing of a company that produces two kinds of drugs. 
The drugs contain a specific active agent, which is extracted from two different kinds of raw materials that should be purchased on the market. 

The goal is to maximize the total profit obtained from producing the drugs, which means minimizing purchasing costs for buying the raw materials and minimizing operational costs for producing the drugs while maximizing sales. 

You are given a budget for purchasing raw materials and operating the production process of the drugs which cannot be exceeded. 
Additionally, you need to keep the capacity constraints for the production of the drugs in mind as there is only a limited amount of manpower as well as equipment hours available. 
Your company can also only store a limited amount of raw materials. 
Most importantly, the amount of active ingredient extracted from the raw materials you purchase needs to satisfy the required amount of active ingredient in the drugs your company produces. 
Importantly, your experience with the production of the drugs has shown the the amount of active ingredient your process extracts from the two raw materials can vary by up to 0.5% and 2%, respectively. 
During purchasing, you need to make sure that no matter how much the amount of active ingredient varies, the required amount for the drug production is always met. '''

## 2. Generate the mathematical model

In [209]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [210]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Drug Production

**Parameters:**

* **Raw Materials:**
    * $R$: Set of raw materials, indexed by $r$ ($r = 1, 2$).
    * $P_r$: Price per unit of raw material $r$.
    * $C_r$: Capacity of storage for raw material $r$.
    * $E_r$:  Average amount of active ingredient extracted per unit of raw material $r$.
    * $V_r$: Variability in active ingredient extraction from raw material $r$ (0.005 for $r=1$, 0.02 for $r=2$).

* **Drugs:**
    * $D$: Set of drugs, indexed by $d$ ($d=1, 2$).
    * $S_d$: Selling price per unit of drug $d$.
    * $Q_d$: Required amount of active ingredient per unit of drug $d$.
    * $L_d$: Labor hours required to produce one unit of drug $d$.
    * $M_d$: Machine hours required to produce one unit of drug $d$.

* **Production & Resources:**
    * $B$: Total budget for raw materials and production.
    * $L$: Total available labor hours.
    * $M$: Total available machine hours. 

**Decision Variables:**

* $X_r$: Amount of raw material $r$ to purchase.
* $Y_d$: Amount of drug $d$ to produce.

**Objective Function (Maximize Profit):**

```
Maximize:  ∑_(d∈D) (S_d * Y_d) - ∑_(r∈R) (P_r * X_r) 
```

**Constraints:**

1. **Budget Constraint:**
   ```
   ∑_(r∈R) (P_r * X_r) + ∑_(d∈D) (L_d * Y_d) + ∑_(d∈D) (M_d * Y_d)  ≤ B
   ```

2. **Storage Capacity Constraint:**
   ```
   X_r ≤ C_r,  ∀ r ∈ R
   ```

3. **Labor Hour Constraint:**
   ```
   ∑_(d∈D) (L_d * Y_d) ≤ L
   ```

4. **Machine Hour Constraint:**
   ```
   ∑_(d∈D) (M_d * Y_d) ≤ M
   ```

5. **Active Ingredient Requirement Constraint (considering variability):**
   ```
   E_r * (1 - V_r) * X_r  ≥ ∑_(d∈D) (Q_d * Y_d) , ∀ r ∈ R
   ```

**Explanation of Constraints:**

* **Constraint 1** ensures that the total cost of raw materials and production does not exceed the allocated budget.
* **Constraint 2** ensures that the purchased amount of each raw material does not exceed the storage capacity.
* **Constraint 3 & 4** ensure that the production of drugs does not exceed the available labor and machine hours.
* **Constraint 5** is crucial and ensures that the amount of active ingredient extracted from the purchased raw materials is sufficient to produce the planned quantity of drugs, even considering the worst-case variability in extraction efficiency.

This model aims to find the optimal purchasing and production plan that maximizes profit while adhering to all budget, resource, and active ingredient requirements. 


## 3. Generate the pyomo code

In [211]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [212]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with actual data
# Raw Materials
R = [1, 2]  
P = {1: 50, 2: 70}  # Price per unit
C = {1: 1000, 2: 800}  # Storage capacity
E = {1: 0.8, 2: 0.9}  # Active ingredient extraction rate
V = {1: 0.005, 2: 0.02}  # Variability in extraction

# Drugs
D = [1, 2]  
S = {1: 200, 2: 250}  # Selling price per unit
Q = {1: 0.5, 2: 0.6}  # Active ingredient required per unit
L = {1: 2, 2: 3}  # Labor hours per unit
M = {1: 1, 2: 1.5}  # Machine hours per unit

# Resources
B = 100000  # Total budget
Labor_hours = 5000  # Total labor hours
Machine_hours = 3000  # Total machine hours 

# Model Creation
model = pyo.ConcreteModel()

# Decision Variables
model.X = pyo.Var(R, within=pyo.NonNegativeReals) # Amount of raw material r to purchase
model.Y = pyo.Var(D, within=pyo.NonNegativeReals) # Amount of drug d to produce

# Objective Function
model.profit = pyo.Objective(expr = sum(S[d]*model.Y[d] for d in D) - sum(P[r]*model.X[r] for r in R), 
                             sense=pyo.maximize) 

# Constraints
model.budget_constraint = pyo.Constraint(expr = sum(P[r]*model.X[r] for r in R) + sum(L[d]*model.Y[d] for d in D) 
                                              + sum(M[d]*model.Y[d] for d in D)  <= B)

model.storage_constraint = pyo.ConstraintList()
for r in R:
    model.storage_constraint.add(model.X[r] <= C[r])

model.labor_constraint = pyo.Constraint(expr = sum(L[d]*model.Y[d] for d in D) <= Labor_hours)

model.machine_constraint = pyo.Constraint(expr = sum(M[d]*model.Y[d] for d in D) <= Machine_hours)

model.ingredient_constraint = pyo.ConstraintList()
for r in R:
    model.ingredient_constraint.add(E[r] * (1 - V[r]) * model.X[r]  >= sum(Q[d] * model.Y[d] for d in D))

# Solve the model
solver = pyo.SolverFactory('glpk') # Choose a solver, e.g., 'glpk', 'gurobi', 'cplex'
results = solver.solve(model)

# Display Results
print(results)
print("Profit =", pyo.value(model.profit))
print("Raw Material Purchase:")
for r in R:
    print(f"X[{r}] = {pyo.value(model.X[r])}")
print("Drug Production:")
for d in D:
    print(f"Y[{d}] = {pyo.value(model.Y[d])}")
```

**Explanation:**

1.  **Import Pyomo:** `import pyomo.environ as pyo` imports the Pyomo library for optimization modeling.
2.  **Sample Data:** The code provides sample data for raw materials, drugs, and resources. You should replace these with your actual data.
3.  **Model Creation:** `model = pyo.ConcreteModel()` creates a ConcreteModel object, which is a specific type of Pyomo model for concrete problems.
4.  **Decision Variables:** 
    *   `model.X = pyo.Var(R, within=pyo.NonNegativeReals)` defines the decision variable for the amount of each raw material to purchase, ensuring it's non-negative.
    *   `model.Y = pyo.Var(D, within=pyo.NonNegativeReals)` defines the decision variable for the amount of each drug to produce, also ensuring it's non-negative.
5.  **Objective Function:** `model.profit` defines the objective function to maximize the profit, which is the total revenue from drug sales minus the total cost of raw materials.
6.  **Constraints:**  Each constraint is defined using `pyo.Constraint` or `pyo.ConstraintList` (for multiple constraints of the same type):
    *   `model.budget_constraint` ensures the total cost stays within the budget.
    *   `model.storage_constraint` ensures each raw material purchase doesn't exceed its storage capacity.
    *   `model.labor_constraint` ensures production doesn't exceed available labor hours.
    *   `model.machine_constraint` ensures production doesn't exceed available machine hours.
    *   `model.ingredient_constraint` ensures sufficient active ingredients are available for drug production, considering the worst-case variability.
7.  **Solving the Model:**
    *   `solver = pyo.SolverFactory('glpk')` chooses a solver. You can replace 'glpk' with other solvers like 'gurobi' or 'cplex' if they are installed.
    *   `results = solver.solve(model)` solves the model using the chosen solver.
8.  **Displaying Results:** The code prints the solver's status (`results`), the optimal profit, the amount of each raw material to purchase (`model.X`), and the amount of each drug to produce (`model.Y`).

This code provides a basic implementation of the drug production optimization model using Pyomo. You can modify and extend this model to include additional constraints, variables, or other aspects relevant to your specific scenario.


## 4. Input problem data and try running the generated code

In [213]:
import pyomo.environ as pyo

# raw_materials = {'RM1', 'RM2'}
# drugs = {'DrugA', 'DrugB'}

# price_rm = {'RM1': 100, 'RM2': 199.90}  
# capacity_rm = {'RM1': 500, 'RM2': 500} 
# extraction_rate = {'RM1': 0.01, 'RM2': 0.02}
# deviation_extraction = {'RM1': 0.005, 'RM2': 0.02}

# selling_price = {'DrugA': 6200, 'DrugB': 6900}
# active_ingredient_req = {'DrugA': 0.5, 'DrugB': 0.6}
# labor_hours = {'DrugA': 90, 'DrugB': 100}
# machine_hours = {'DrugA': 40, 'DrugB': 50}

# budget = 100000
# total_labor_hours = 2000
# total_machine_hours = 800

# Sample Data - Replace with actual data
# Raw Materials
R = [1, 2]  
P = {1: 100, 2: 199.90}  # Price per unit
C = {1: 1000, 2: 800}  # Storage capacity
E = {1: 0.01, 2: 0.02}  # Active ingredient extraction rate
V = {1: 0.005, 2: 0.02}  # Variability in extraction

# Drugs
D = [1, 2]  
S = {1: 6200, 2: 6900}  # Selling price per unit
Q = {1: 0.5, 2: 0.6}  # Active ingredient required per unit
L = {1: 90, 2: 100}  # Labor hours per unit
M = {1: 40, 2: 50}  # Machine hours per unit

# Resources
B = 100000  # Total budget
Labor_hours = 2000  # Total labor hours
Machine_hours = 800  # Total machine hours 

# Model Creation
model = pyo.ConcreteModel()

# Decision Variables
model.X = pyo.Var(R, within=pyo.NonNegativeReals) # Amount of raw material r to purchase
model.Y = pyo.Var(D, within=pyo.NonNegativeReals) # Amount of drug d to produce

# Objective Function
model.profit = pyo.Objective(expr = sum(S[d]*model.Y[d] for d in D) - sum(P[r]*model.X[r] for r in R), 
                             sense=pyo.maximize) 

# Constraints
model.budget_constraint = pyo.Constraint(expr = sum(P[r]*model.X[r] for r in R) + sum(L[d]*model.Y[d] for d in D) 
                                              + sum(M[d]*model.Y[d] for d in D)  <= B)

model.storage_constraint = pyo.ConstraintList()
for r in R:
    model.storage_constraint.add(model.X[r] <= C[r])

model.labor_constraint = pyo.Constraint(expr = sum(L[d]*model.Y[d] for d in D) <= Labor_hours)

model.machine_constraint = pyo.Constraint(expr = sum(M[d]*model.Y[d] for d in D) <= Machine_hours)

model.ingredient_constraint = pyo.ConstraintList()
for r in R:
    model.ingredient_constraint.add(E[r] * (1 - V[r]) * model.X[r]  >= sum(Q[d] * model.Y[d] for d in D))

# Solve the model
solver = pyo.SolverFactory('glpk') # Choose a solver, e.g., 'glpk', 'gurobi', 'cplex'
results = solver.solve(model)

# Display Results
print(results)
print("Profit =", pyo.value(model.profit))
print("Raw Material Purchase:")
for r in R:
    print(f"X[{r}] = {pyo.value(model.X[r])}")
print("Drug Production:")
for d in D:
    print(f"Y[{d}] = {pyo.value(model.Y[d])}")


Problem: 
- Name: unknown
  Lower bound: 0.0
  Upper bound: 0.0
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 4
  Number of nonzeros: 16
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.039862871170043945
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Profit = 0.0
Raw Material Purchase:
X[1] = 0.0
X[2] = 0.0
Drug Production:
Y[1] = 0.0
Y[2] = 0.0


In [215]:
model.pprint()

2 Var Declarations
    X : Size=2, Index={1, 2}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :  None : False : False : NonNegativeReals
          2 :     0 :   0.0 :  None : False : False : NonNegativeReals
    Y : Size=2, Index={1, 2}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :  None : False : False : NonNegativeReals
          2 :     0 :   0.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 6200*Y[1] + 6900*Y[2] - (100*X[1] + 199.9*X[2])

5 Constraint Declarations
    budget_constraint : Size=1, Index=None, Active=True
        Key  : Lower : Body                                                           : Upper    : Active
        None :  -Inf : 100*X[1] + 199.9*X[2] + 90*Y[1] + 100*Y[2] + 40*Y[1] + 50*Y[2] : 100000.0 :   True
    ingredient_constra

## 5. Correct the code to verify model viability (optional)

In [216]:
import pyomo.environ as pyo

# Parameters
p1, p2 = 6200, 6900  # Prices of drugs
c1, c2, c3, c4 = 100, 199.90, 700, 800 # Costs
a1, a2, a3, a4 = 0.01, 0.02, 0.500, 0.600  # Active ingredient extraction coefficients
t1M, t2M, t1E, t2E = 90.0, 100.0, 40.0, 50.0  # Time requirements
MPH, EH = 2000, 800  # Manpower and equipment hours limits
S, B = 1000, 100000  # Storage and budget limits

# Create a model instance
model = pyo.ConcreteModel()

# Declare decision variables
model.d1 = pyo.Var(domain=pyo.NonNegativeReals)
model.d2 = pyo.Var(domain=pyo.NonNegativeReals)
model.r1 = pyo.Var(domain=pyo.NonNegativeReals)
model.r2 = pyo.Var(domain=pyo.NonNegativeReals)

# Objective Function
model.obj = pyo.Objective(expr=(p1*model.d1 + p2*model.d2 - 
                                (c1*model.r1 + c2*model.r2 + c3*model.d1 + c4*model.d2)), sense=pyo.maximize)

# Constraints
model.active_ingredient = pyo.Constraint(expr=(0.995*a1*model.r1 + 0.98*a2*model.r2 - a3*model.d1 - a4*model.d2 >= 0))
model.manpower = pyo.Constraint(expr=(t1M*model.d1 + t2M*model.d2 <= MPH))
model.equipment = pyo.Constraint(expr=(t1E*model.d1 + t2E*model.d2 <= EH))
model.storage = pyo.Constraint(expr=(model.r1 + model.r2 <= S))
model.budget = pyo.Constraint(expr=(c1*model.r1 + c2*model.r2 + c3*model.d1 + c4*model.d2 <= B))

# Solve the model
solver = pyo.SolverFactory('glpk')
result = solver.solve(model)

# Display the results
print(f"Status: {result.solver.status}")
print(f"Objective value: {pyo.value(model.obj)}")
print(f"d1 = {pyo.value(model.d1)}")
print(f"d2 = {pyo.value(model.d2)}")
print(f"r1 = {pyo.value(model.r1)}")
print(f"r2 = {pyo.value(model.r2)}")
     

Status: ok
Objective value: 8294.566839287349
d1 = 17.4668656192399
d2 = 0.0
r1 = 877.731940665321
r2 = 0.0


In [217]:
model.pprint()

4 Var Declarations
    d1 : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 17.4668656192399 :  None : False : False : NonNegativeReals
    d2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    r1 : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 877.731940665321 :  None : False : False : NonNegativeReals
    r2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 6200*d1 + 6900*d2 - (100*r1 + 199.9*r2 + 700*d1 + 800*d2)

5 Constraint Declarations
    active_ingredient : Size=1, Index=None, Active=True

## 6. Printing the outputs as strings, so they can be saved.

In [218]:
print(response.text)

## Mathematical Optimization Model for Drug Production

**Parameters:**

* **Raw Materials:**
    * $R$: Set of raw materials, indexed by $r$ ($r = 1, 2$).
    * $P_r$: Price per unit of raw material $r$.
    * $C_r$: Capacity of storage for raw material $r$.
    * $E_r$:  Average amount of active ingredient extracted per unit of raw material $r$.
    * $V_r$: Variability in active ingredient extraction from raw material $r$ (0.005 for $r=1$, 0.02 for $r=2$).

* **Drugs:**
    * $D$: Set of drugs, indexed by $d$ ($d=1, 2$).
    * $S_d$: Selling price per unit of drug $d$.
    * $Q_d$: Required amount of active ingredient per unit of drug $d$.
    * $L_d$: Labor hours required to produce one unit of drug $d$.
    * $M_d$: Machine hours required to produce one unit of drug $d$.

* **Production & Resources:**
    * $B$: Total budget for raw materials and production.
    * $L$: Total available labor hours.
    * $M$: Total available machine hours. 

**Decision Variables:**

* $X_r$: Amount

In [219]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with actual data
# Raw Materials
R = [1, 2]  
P = {1: 50, 2: 70}  # Price per unit
C = {1: 1000, 2: 800}  # Storage capacity
E = {1: 0.8, 2: 0.9}  # Active ingredient extraction rate
V = {1: 0.005, 2: 0.02}  # Variability in extraction

# Drugs
D = [1, 2]  
S = {1: 200, 2: 250}  # Selling price per unit
Q = {1: 0.5, 2: 0.6}  # Active ingredient required per unit
L = {1: 2, 2: 3}  # Labor hours per unit
M = {1: 1, 2: 1.5}  # Machine hours per unit

# Resources
B = 100000  # Total budget
Labor_hours = 5000  # Total labor hours
Machine_hours = 3000  # Total machine hours 

# Model Creation
model = pyo.ConcreteModel()

# Decision Variables
model.X = pyo.Var(R, within=pyo.NonNegativeReals) # Amount of raw material r to purchase
model.Y = pyo.Var(D, within=pyo.NonNegativeReals) # Amount of drug d to produce

# Objective Function
model.profit = pyo.Objective(expr = sum(S[d]*model.Y[d] for d in D) - sum(P[r]*model.X[r] for r in R),